In [9]:
# 2) 유튜브 공범 컨텐츠동영상 데이터
import pandas as pd

"각 데이터의 ‘ct’컬럼을 시간으로 인식할수 있게 datatype을 변경하고 video 데이터의 videoname의 각 value 마다 몇개의 데이터씩 가지고 있는지 확인하라"
channel =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/channelInfo.csv')
video =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/videoInfo.csv')

channel.ct = pd.to_datetime(channel.ct)
video.ct = pd.to_datetime(video.ct)
#print(video.groupby(["videoname"])["videoname"].count())

"수집된 각 video의 가장 최신화 된 날짜의 viewcount값을 출력하라"
print(video.head())
#print(video.groupby(["videopk"])["viewcnt"].max())


       videopk  viewcnt  likecnt  dislikecnt  favoritecnt  cmcnt  \
0  c5JQp6xafqc  1667010    30474         706            0   6587   
1  c5JQp6xafqc  1669089    30495         707            0   6589   
2  c5JQp6xafqc  1674759    30522         711            0   6596   
3  c5JQp6xafqc  1677026    30555         712            0   6604   
4  c5JQp6xafqc  1681824    30585         713            0   6600   

                   ct videoname  
0 2021-10-10 15:20:03    공범 EP1  
1 2021-10-10 15:30:03    공범 EP1  
2 2021-10-10 15:40:02    공범 EP1  
3 2021-10-10 15:50:03    공범 EP1  
4 2021-10-10 16:00:03    공범 EP1  
